In [14]:
import pandas as pd
import numpy as np

from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
import matplotlib.pyplot as plt
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.model_selection import RandomizedSearchCV
from sklearn.metrics import RocCurveDisplay
from sklearn.metrics import accuracy_score, classification_report
pd.set_option('display.max_columns',None)
import joblib

In [18]:
test_day = pd.read_parquet(f'C:/Users/Samsung/Documents/GitHub-Datas/TCC_Machine_Learning/Datas referências/price_data_D20.parquet')
test_day.head()

,Date,Open,High,Low,Close,Adj Close,Volume,Ticker,change_in_price,Smoothed_Close,Prediction,Smoothed_Close_1,Verify,Close_20,Smoothed_Close_20,down_days,up_days,RSI,low_14,high_14,k_percent,r_percent,MACD,MACD_EMA,Price_Rate_Of_Change,On Balance Volume,Match
0,2000-02-22,0.485255,0.485255,0.485255,0.485255,0.260847,75,ABEV3.SA,-0.029034,0.515470,1.0,0.485255,1.0,0.639548,0.528984,0.029034,0.000000,43.244009,0.468107,0.527474,79.779904,-20.220096,-0.000492,-0.000328,-0.005624,13882,1
1,2000-02-23,0.494478,0.494478,0.494478,0.494478,0.265805,455,ABEV3.SA,0.009223,0.515050,1.0,0.494478,1.0,0.657994,0.531564,0.000000,0.009223,47.299662,0.468107,0.527474,79.072712,-20.927288,-0.000553,-0.000375,-0.006782,13427,1
2,2000-02-24,0.487885,0.487885,0.487885,0.487885,0.262261,5005,ABEV3.SA,-0.006593,0.514507,1.0,0.487885,1.0,0.652734,0.533988,0.006593,0.000000,44.667003,0.468107,0.527474,78.157553,-21.842447,-0.000626,-0.000427,-0.008044,8422,1
3,2000-02-25,0.474700,0.474700,0.474700,0.474700,0.255173,3033,ABEV3.SA,-0.013185,0.513711,1.0,0.474700,1.0,0.652734,0.536362,0.013185,0.000000,39.583155,0.468107,0.527474,76.816512,-23.183488,-0.000723,-0.000487,-0.008907,5389,1
4,2000-02-29,0.468107,0.468107,0.468107,0.468107,0.251629,11602,ABEV3.SA,-0.006593,0.512799,1.0,0.468107,1.0,0.606585,0.537767,0.006593,0.000000,37.143972,0.468107,0.527474,75.280182,-24.719818,-0.000846,-0.000560,-0.008746,-6213,1


In [19]:
test_day['Ticker'].unique()

array(['ABEV3.SA'], dtype=object)

In [20]:
test_day_AMBEV = test_day[test_day['Ticker'] == 'ABEV3.SA'].reset_index(drop=True)
test_day_AMBEV['Prediction'] = np.NaN

test_day_AMBEV.head()

,Date,Open,High,Low,Close,Adj Close,Volume,Ticker,change_in_price,Smoothed_Close,Prediction,Smoothed_Close_1,Verify,Close_20,Smoothed_Close_20,down_days,up_days,RSI,low_14,high_14,k_percent,r_percent,MACD,MACD_EMA,Price_Rate_Of_Change,On Balance Volume,Match
0,2000-02-22,0.485255,0.485255,0.485255,0.485255,0.260847,75,ABEV3.SA,-0.029034,0.515470,NaN,0.485255,1.0,0.639548,0.528984,0.029034,0.000000,43.244009,0.468107,0.527474,79.779904,-20.220096,-0.000492,-0.000328,-0.005624,13882,1
1,2000-02-23,0.494478,0.494478,0.494478,0.494478,0.265805,455,ABEV3.SA,0.009223,0.515050,NaN,0.494478,1.0,0.657994,0.531564,0.000000,0.009223,47.299662,0.468107,0.527474,79.072712,-20.927288,-0.000553,-0.000375,-0.006782,13427,1
2,2000-02-24,0.487885,0.487885,0.487885,0.487885,0.262261,5005,ABEV3.SA,-0.006593,0.514507,NaN,0.487885,1.0,0.652734,0.533988,0.006593,0.000000,44.667003,0.468107,0.527474,78.157553,-21.842447,-0.000626,-0.000427,-0.008044,8422,1
3,2000-02-25,0.474700,0.474700,0.474700,0.474700,0.255173,3033,ABEV3.SA,-0.013185,0.513711,NaN,0.474700,1.0,0.652734,0.536362,0.013185,0.000000,39.583155,0.468107,0.527474,76.816512,-23.183488,-0.000723,-0.000487,-0.008907,5389,1
4,2000-02-29,0.468107,0.468107,0.468107,0.468107,0.251629,11602,ABEV3.SA,-0.006593,0.512799,NaN,0.468107,1.0,0.606585,0.537767,0.006593,0.000000,37.143972,0.468107,0.527474,75.280182,-24.719818,-0.000846,-0.000560,-0.008746,-6213,1


In [21]:
test_day_AMBEV['Date'] = pd.to_datetime(test_day_AMBEV['Date'])

# Definir a data inicial para filtrar
inicio = '2000-02-22'  # Data inicial

# Filtrar as linhas onde a data é maior ou igual a 'inicio'
test_day_AMBEV = test_day_AMBEV[test_day_AMBEV['Date'] >= inicio]

test_day_AMBEV.head(15)


,Date,Open,High,Low,Close,Adj Close,Volume,Ticker,change_in_price,Smoothed_Close,Prediction,Smoothed_Close_1,Verify,Close_20,Smoothed_Close_20,down_days,up_days,RSI,low_14,high_14,k_percent,r_percent,MACD,MACD_EMA,Price_Rate_Of_Change,On Balance Volume,Match
0,2000-02-22,0.485255,0.485255,0.485255,0.485255,0.260847,75,ABEV3.SA,-0.029034,0.515470,NaN,0.485255,1.0,0.639548,0.528984,0.029034,0.000000,43.244009,0.468107,0.527474,79.779904,-20.220096,-0.000492,-0.000328,-0.005624,13882,1
1,2000-02-23,0.494478,0.494478,0.494478,0.494478,0.265805,455,ABEV3.SA,0.009223,0.515050,NaN,0.494478,1.0,0.657994,0.531564,0.000000,0.009223,47.299662,0.468107,0.527474,79.072712,-20.927288,-0.000553,-0.000375,-0.006782,13427,1
2,2000-02-24,0.487885,0.487885,0.487885,0.487885,0.262261,5005,ABEV3.SA,-0.006593,0.514507,NaN,0.487885,1.0,0.652734,0.533988,0.006593,0.000000,44.667003,0.468107,0.527474,78.157553,-21.842447,-0.000626,-0.000427,-0.008044,8422,1
3,2000-02-25,0.474700,0.474700,0.474700,0.474700,0.255173,3033,ABEV3.SA,-0.013185,0.513711,NaN,0.474700,1.0,0.652734,0.536362,0.013185,0.000000,39.583155,0.468107,0.527474,76.816512,-23.183488,-0.000723,-0.000487,-0.008907,5389,1
4,2000-02-29,0.468107,0.468107,0.468107,0.468107,0.251629,11602,ABEV3.SA,-0.006593,0.512799,NaN,0.468107,1.0,0.606585,0.537767,0.006593,0.000000,37.143972,0.468107,0.527474,75.280182,-24.719818,-0.000846,-0.000560,-0.008746,-6213,1
5,2000-03-03,0.512957,0.512957,0.512957,0.512957,0.275738,758,ABEV3.SA,0.044850,0.512802,NaN,0.512957,1.0,0.626363,0.539539,0.000000,0.044850,57.635161,0.468107,0.527474,75.285518,-24.714482,-0.000930,-0.000635,-0.006850,-5455,1
6,2000-03-09,0.527474,0.527474,0.527474,0.527474,0.283542,1971,ABEV3.SA,0.014517,0.513095,NaN,0.527474,1.0,0.626363,0.541275,0.000000,0.014517,62.233396,0.468107,0.527474,75.779808,-24.220192,-0.000964,-0.000702,-0.006457,-3484,1
7,2000-03-14,0.494478,0.494478,0.494478,0.494478,0.265805,227,ABEV3.SA,-0.032996,0.512723,NaN,0.494478,1.0,0.619771,0.542845,0.032996,0.000000,48.443694,0.468107,0.527474,75.152617,-24.847383,-0.001004,-0.000763,-0.006588,-3711,1
8,2000-03-17,0.501104,0.501104,0.501104,0.501104,0.269367,1516,ABEV3.SA,0.006626,0.512490,NaN,0.501104,1.0,0.593400,0.543856,0.000000,0.006626,50.961417,0.468107,0.527474,74.761192,-25.238808,-0.001038,-0.000818,-0.006968,-5227,1
9,2000-03-22,0.520882,0.520882,0.520882,0.520882,0.279998,7583,ABEV3.SA,0.019778,0.512658,NaN,0.520882,1.0,0.586808,0.544715,0.000000,0.019778,58.021804,0.468107,0.527474,75.043893,-24.956107,-0.001041,-0.000863,-0.005454,2356,1


In [22]:
# Carregar o modelo salvo
rand_frst_clf_Ambev = joblib.load('C:/Users/Samsung/Documents/GitHub-Datas/TCC_Machine_Learning/random_forest_model_Ambev.pkl')

In [23]:
Features = test_day_AMBEV[['RSI','k_percent','r_percent','Price_Rate_Of_Change','MACD','MACD_EMA','On Balance Volume']]
Features.head()

,RSI,k_percent,r_percent,Price_Rate_Of_Change,MACD,MACD_EMA,On Balance Volume
0,43.244009,79.779904,-20.220096,-0.005624,-0.000492,-0.000328,13882
1,47.299662,79.072712,-20.927288,-0.006782,-0.000553,-0.000375,13427
2,44.667003,78.157553,-21.842447,-0.008044,-0.000626,-0.000427,8422
3,39.583155,76.816512,-23.183488,-0.008907,-0.000723,-0.000487,5389
4,37.143972,75.280182,-24.719818,-0.008746,-0.000846,-0.000560,-6213


In [24]:
Features_array = Features.to_numpy()

# Fazer previsões usando o modelo carregado
predictions = rand_frst_clf_Ambev.predict(Features_array)

# Adicionar as previsões em uma nova coluna chamada 'Prediction'
test_day_AMBEV['Prediction'] = predictions

# Visualizar o DataFrame com a nova coluna
test_day_AMBEV.head()

c:\ProgramData\anaconda3\lib\site-packages\sklearn\base.py:465: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(


,Date,Open,High,Low,Close,Adj Close,Volume,Ticker,change_in_price,Smoothed_Close,Prediction,Smoothed_Close_1,Verify,Close_20,Smoothed_Close_20,down_days,up_days,RSI,low_14,high_14,k_percent,r_percent,MACD,MACD_EMA,Price_Rate_Of_Change,On Balance Volume,Match
0,2000-02-22,0.485255,0.485255,0.485255,0.485255,0.260847,75,ABEV3.SA,-0.029034,0.515470,1.0,0.485255,1.0,0.639548,0.528984,0.029034,0.000000,43.244009,0.468107,0.527474,79.779904,-20.220096,-0.000492,-0.000328,-0.005624,13882,1
1,2000-02-23,0.494478,0.494478,0.494478,0.494478,0.265805,455,ABEV3.SA,0.009223,0.515050,1.0,0.494478,1.0,0.657994,0.531564,0.000000,0.009223,47.299662,0.468107,0.527474,79.072712,-20.927288,-0.000553,-0.000375,-0.006782,13427,1
2,2000-02-24,0.487885,0.487885,0.487885,0.487885,0.262261,5005,ABEV3.SA,-0.006593,0.514507,1.0,0.487885,1.0,0.652734,0.533988,0.006593,0.000000,44.667003,0.468107,0.527474,78.157553,-21.842447,-0.000626,-0.000427,-0.008044,8422,1
3,2000-02-25,0.474700,0.474700,0.474700,0.474700,0.255173,3033,ABEV3.SA,-0.013185,0.513711,1.0,0.474700,1.0,0.652734,0.536362,0.013185,0.000000,39.583155,0.468107,0.527474,76.816512,-23.183488,-0.000723,-0.000487,-0.008907,5389,1
4,2000-02-29,0.468107,0.468107,0.468107,0.468107,0.251629,11602,ABEV3.SA,-0.006593,0.512799,1.0,0.468107,1.0,0.606585,0.537767,0.006593,0.000000,37.143972,0.468107,0.527474,75.280182,-24.719818,-0.000846,-0.000560,-0.008746,-6213,1


In [25]:
test_day_AMBEV.to_excel('C:/Users/Samsung/Documents/GitHub-Datas/TCC_Machine_Learning/Datas referências/test_day.xlsx', index=False)